<a href="https://colab.research.google.com/github/DU-ds/pyspark_udemy/blob/main/Section2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! apt update
! apt install openjdk-8-jdk-headless -qq > /dev/null
! wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
! tar xf spark-2.3.1-bin-hadoop2.7.tgz
! pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

! ls

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:15

In [3]:
spark = SparkSession.builder.getOrCreate()
spark

In [5]:

data = spark.read.format("csv").option("header", "true").load("original.csv")

In [6]:
data.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [7]:
from pyspark.sql.functions import *
data = data.withColumn("city", when(data.City.isNull(), "Unknown").otherwise(data.City)) 

In [8]:
data.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           city|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|        Unknown|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [9]:
data = data.filter(data.JobTitle.isNotNull())

In [ ]:
data.show()

In [11]:
data = data.withColumn("clean_salary", data.Salary.substr(2,100).cast('float'))
data.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+------------+
| id|first_name| last_name|gender|           city|            JobTitle|   Salary|  Latitude|  Longitude|clean_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|    57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|     62846.6|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|    61489.23|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|    63863.09|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|    30101.16|
|  8|   Goddart|     Flear|  Male|      Trélissac|Deskto

In [12]:
mean = data.groupBy().avg("clean_salary")
mean.show()

+-----------------+
|avg(clean_salary)|
+-----------------+
|55516.32088199837|
+-----------------+



In [13]:
mean.take(1)

[Row(avg(clean_salary)=55516.32088199837)]

In [14]:
mean.take(1)[0]

Row(avg(clean_salary)=55516.32088199837)

In [15]:
avg = mean.take(1)[0][0]
avg

55516.32088199837

In [16]:
data = data.withColumn('new_salary', when(data.clean_salary.isNull(), lit(avg)).otherwise(data.clean_salary))
data.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+------------+----------------+
| id|first_name| last_name|gender|           city|            JobTitle|   Salary|  Latitude|  Longitude|clean_salary|      new_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+------------+----------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|    57438.18|   57438.1796875|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|     62846.6|   62846.6015625|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|    61489.23|  61489.23046875|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|    63863.09|  63863.08984375|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|     

In [22]:
import numpy as np
latitudes = data.select('Latitude')
latitudes.show()


+----------+
|  Latitude|
+----------+
|50.5774075|
|48.8231572|
|44.5047212|
|      null|
|53.4266145|
|45.1905186|
| 32.027934|
|  4.272793|
|     -5.85|
| 39.172378|
|49.8151822|
|42.1014803|
|49.7923299|
|43.4945737|
|52.7441662|
| 38.696249|
|-7.7232567|
|40.7172049|
|  49.16291|
|40.7576842|
+----------+
only showing top 20 rows



In [24]:
latitudes = latitudes.filter(latitudes.Latitude.isNotNull())
latitudes.show()

+----------+
|  Latitude|
+----------+
|50.5774075|
|48.8231572|
|44.5047212|
|53.4266145|
|45.1905186|
| 32.027934|
|  4.272793|
|     -5.85|
| 39.172378|
|49.8151822|
|42.1014803|
|49.7923299|
|43.4945737|
|52.7441662|
| 38.696249|
|-7.7232567|
|40.7172049|
|  49.16291|
|40.7576842|
|48.4902808|
+----------+
only showing top 20 rows



In [25]:
# import folium
# folium.Map()

In [28]:
latitudes = latitudes.withColumn('latitude2', latitudes.Latitude.cast("float"))
latitudes.show()

+----------+----------+
|  Latitude| latitude2|
+----------+----------+
|50.5774075| 50.577408|
|48.8231572|  48.82316|
|44.5047212| 44.504723|
|53.4266145| 53.426613|
|45.1905186| 45.190517|
| 32.027934| 32.027935|
|  4.272793|  4.272793|
|     -5.85|     -5.85|
| 39.172378|  39.17238|
|49.8151822|  49.81518|
|42.1014803|  42.10148|
|49.7923299|  49.79233|
|43.4945737| 43.494576|
|52.7441662| 52.744167|
| 38.696249| 38.696247|
|-7.7232567|-7.7232566|
|40.7172049| 40.717205|
|  49.16291|  49.16291|
|40.7576842| 40.757683|
|48.4902808|  48.49028|
+----------+----------+
only showing top 20 rows



In [30]:
latitudes = latitudes.select('latitude2')
latitudes.show()

+----------+
| latitude2|
+----------+
| 50.577408|
|  48.82316|
| 44.504723|
| 53.426613|
| 45.190517|
| 32.027935|
|  4.272793|
|     -5.85|
|  39.17238|
|  49.81518|
|  42.10148|
|  49.79233|
| 43.494576|
| 52.744167|
| 38.696247|
|-7.7232566|
| 40.717205|
|  49.16291|
| 40.757683|
|  48.49028|
+----------+
only showing top 20 rows



In [31]:
median = np.median(latitudes.collect())

In [32]:
print(median)

31.93397331237793


In [35]:
data = data.withColumn('lat', when(data.Latitude.isNull(), lit(median)).otherwise(data.Latitude) )

In [36]:
data.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+------------+----------------+-----------------+
| id|first_name| last_name|gender|           city|            JobTitle|   Salary|  Latitude|  Longitude|clean_salary|      new_salary|              lat|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+------------+----------------+-----------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|    57438.18|   57438.1796875|       50.5774075|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|     62846.6|   62846.6015625|       48.8231572|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|    61489.23|  61489.23046875|       44.5047212|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09| 

In [37]:
from pyspark.sql import functions as sqlf
genders = data.groupBy("gender").agg(sqlf.avg("new_salary").alias("AvgSalary"))

In [38]:
genders.show()

+------+------------------+
|gender|         AvgSalary|
+------+------------------+
|Female|55677.250125558036|
|  Male| 55361.09385573019|
+------+------------------+



In [39]:
df = data.withColumn("female_salary", when(data.gender == "Female", data.new_salary).otherwise(lit(0)))
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+------------+----------------+-----------------+----------------+
| id|first_name| last_name|gender|           city|            JobTitle|   Salary|  Latitude|  Longitude|clean_salary|      new_salary|              lat|   female_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+------------+----------------+-----------------+----------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|    57438.18|   57438.1796875|       50.5774075|   57438.1796875|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|     62846.6|   62846.6015625|       48.8231572|   62846.6015625|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|    61489.23|  61489.23046875|       44.504721

In [40]:
df = df.withColumn("male_salary", when(data.gender == "Male", data.new_salary).otherwise(lit(0)))
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+------------+----------------+-----------------+----------------+----------------+
| id|first_name| last_name|gender|           city|            JobTitle|   Salary|  Latitude|  Longitude|clean_salary|      new_salary|              lat|   female_salary|     male_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+------------+----------------+-----------------+----------------+----------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|    57438.18|   57438.1796875|       50.5774075|   57438.1796875|             0.0|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|     62846.6|   62846.6015625|       48.8231572|   62846.6015625|             0.0|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounti

In [42]:
df.groupBy("JobTitle").agg(sqlf.avg("female_salary").alias("role_female_salary"), sqlf.avg("male_salary").alias("role_male_salary")).show()

+--------------------+------------------+------------------+
|            JobTitle|role_female_salary|  role_male_salary|
+--------------------+------------------+------------------+
|Systems Administr...|   50590.474609375|  15540.9501953125|
|   Media Manager III|29586.436197916668|17381.920572916668|
|  Recruiting Manager|34848.452473958336|  26383.4951171875|
|       Geologist III|      31749.046875|    12830.75390625|
|        Geologist II|               0.0|   43293.865234375|
|Database Administ...|               0.0|     52018.4609375|
|   Financial Analyst|   23353.776953125|       39606.05625|
|  Analyst Programmer|  16406.1287109375|  21042.9634765625|
|Software Engineer II|               0.0|      74782.640625|
|       Accountant IV|   82732.248046875|               0.0|
|    Product Engineer|    41825.48359375|       20464.94375|
|Software Test Eng...|  32218.6083984375|   27122.462890625|
|Safety Technician...|               0.0|   29421.529296875|
|    Junior Executive|15

In [47]:
df = df.groupBy("JobTitle").agg(sqlf.avg("female_salary").alias("role_female_salary"), sqlf.avg("male_salary").alias("role_male_salary"))
df.show()

+--------------------+------------------+------------------+
|            JobTitle|role_female_salary|  role_male_salary|
+--------------------+------------------+------------------+
|Systems Administr...|   50590.474609375|  15540.9501953125|
|   Media Manager III|29586.436197916668|17381.920572916668|
|  Recruiting Manager|34848.452473958336|  26383.4951171875|
|       Geologist III|      31749.046875|    12830.75390625|
|        Geologist II|               0.0|   43293.865234375|
|Database Administ...|               0.0|     52018.4609375|
|   Financial Analyst|   23353.776953125|       39606.05625|
|  Analyst Programmer|  16406.1287109375|  21042.9634765625|
|Software Engineer II|               0.0|      74782.640625|
|       Accountant IV|   82732.248046875|               0.0|
|    Product Engineer|    41825.48359375|       20464.94375|
|Software Test Eng...|  32218.6083984375|   27122.462890625|
|Safety Technician...|               0.0|   29421.529296875|
|    Junior Executive|15

In [50]:
df = df.withColumn('salary_delta', df.role_female_salary - df.role_male_salary)
df.show()

+--------------------+------------------+------------------+-------------------+
|            JobTitle|role_female_salary|  role_male_salary|       salary_delta|
+--------------------+------------------+------------------+-------------------+
|Systems Administr...|   50590.474609375|  15540.9501953125|   35049.5244140625|
|   Media Manager III|29586.436197916668|17381.920572916668|       12204.515625|
|  Recruiting Manager|34848.452473958336|  26383.4951171875|  8464.957356770836|
|       Geologist III|      31749.046875|    12830.75390625|     18918.29296875|
|        Geologist II|               0.0|   43293.865234375|   -43293.865234375|
|Database Administ...|               0.0|     52018.4609375|     -52018.4609375|
|   Financial Analyst|   23353.776953125|       39606.05625|   -16252.279296875|
|  Analyst Programmer|  16406.1287109375|  21042.9634765625| -4636.834765625001|
|Software Engineer II|               0.0|      74782.640625|      -74782.640625|
|       Accountant IV|   827

In [55]:
df = df.withColumn('percent_salary_delta', when(df.role_male_salary > df.role_female_salary, df.salary_delta / df.role_male_salary * 100).otherwise(df.salary_delta/df.role_female_salary * 100))
df.show()


+--------------------+------------------+------------------+-------------------+--------------------+
|            JobTitle|role_female_salary|  role_male_salary|       salary_delta|percent_salary_delta|
+--------------------+------------------+------------------+-------------------+--------------------+
|Systems Administr...|   50590.474609375|  15540.9501953125|   35049.5244140625|    69.2808768541725|
|   Media Manager III|29586.436197916668|17381.920572916668|       12204.515625|  41.250374135494496|
|  Recruiting Manager|34848.452473958336|  26383.4951171875|  8464.957356770836|  24.290769764013355|
|       Geologist III|      31749.046875|    12830.75390625|     18918.29296875|   59.58696348660073|
|        Geologist II|               0.0|   43293.865234375|   -43293.865234375|              -100.0|
|Database Administ...|               0.0|     52018.4609375|     -52018.4609375|              -100.0|
|   Financial Analyst|   23353.776953125|       39606.05625|   -16252.279296875|  

In [56]:
data.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+------------+----------------+-----------------+
| id|first_name| last_name|gender|           city|            JobTitle|   Salary|  Latitude|  Longitude|clean_salary|      new_salary|              lat|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+------------+----------------+-----------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|    57438.18|   57438.1796875|       50.5774075|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|     62846.6|   62846.6015625|       48.8231572|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|    61489.23|  61489.23046875|       44.5047212|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09| 

In [61]:
df = cityavg = data.groupBy("city").agg(sqlf.avg(data.new_salary).alias("CityAvgSalary")).sort(col('CityAvgSalary').desc())
df.show()

+-----------------+-------------+
|             city|CityAvgSalary|
+-----------------+-------------+
|        Mesopotam|  99948.28125|
|       Zhongcheng| 99942.921875|
|           Caxias|99786.3984375|
|      Karangtawar|99638.9921875|
|        Itabaiana|  99502.15625|
|           Pasian|  99421.34375|
|           Webuye| 99368.546875|
|      Yuktae-dong| 99250.828125|
|           Zinder|  99222.84375|
|   Timiryazevskiy|   99142.9375|
|        Sawahbaru|99013.7109375|
|          Madimba|98737.8671875|
|         Huangshi|  98690.34375|
|          Gharyan|   98679.3125|
|         Yŏnan-ŭp| 98628.609375|
|     Wringinputih|98603.8203125|
|Monte da Boavista|  98586.71875|
|          Klukeng|98439.4921875|
|         Murmashi|  98226.15625|
|        Fox Creek|      98138.0|
+-----------------+-------------+
only showing top 20 rows

